# Tareas
1) [DONE] Desde un notebook, script para llamar a ecobici y guardar alguna info SIMPLE (ej: cuántos slots ocupados) en csv.
2) [DONE] Agregar el secret para usar en github actions
3) [DONE] Configurar la Github Action para que haga un commit (en una branch en particular? O en main?)
4) [DONE] Luego de probar que funciona, agregar el cron a la github action
5) [DONE] Mergear a main el nuevo código
6) Agregar un router y separar la github action en 2 jobs: una para el que inserta nueva info de las stations, y la otra que actualiza las distancias.

In [1]:
import pandas as pd

from internal.platform.environment.environment import set_environment
from internal.gateway.transportBA import transportBA
from internal.repository.transportBA.stations_repository import StationsRepository
from internal.gateway.openrouteservice.client import OpenRouteServiceClient
from internal.gateway.openrouteservice.dataframe_adapter import OpenRouteServiceDataFrameAdapter

In [2]:
set_environment()

In [3]:
def preprocessStations(stations_df):
    return stations_df.explode("groups")

def preprocessStationsStatus(stations_status_df):
    stations_status_df["status_last_reported_utc"] = pd.to_datetime(stations_status_df["last_reported"], unit="s", utc=True)
    return station_status_df

In [4]:
# update stations.csv
transportBAClient = transportBA.Client()
stations_df = transportBAClient.GetStations()
station_status_df = transportBAClient.GetStationStatus()
stationsDao = StationsRepository()

stations_df = preprocessStations(stations_df)
station_status_df = preprocessStationsStatus(station_status_df)
stations_with_status_df = pd.merge(stations_df, station_status_df, on="station_id")

stationsDao.AppendWithJobRunAt(stations_with_status_df)

In [13]:
from internal.repository.transportBA.journeys_repository import JourneysRepository
journeysDao = JourneysRepository()
dataFrameAdapterClient = OpenRouteServiceDataFrameAdapter(client=OpenRouteServiceClient())

length = len(stations_df)
lower, stations_per_req = 0, 59  # max of 59 elements in request
journeys_df = pd.DataFrame()
stationsFromCSV = stationsDao.GroupByStationIDWithCoordinates()
for i in range(0, int(length/59)):
    stations_df_short = stationsFromCSV.iloc[lower*stations_per_req:(lower+1)*stations_per_req]
    df = dataFrameAdapterClient.GetDurationMatrixFromDataFrame("cycling-regular", stations_df_short)
    journeys_df = pd.concat([journeys_df, df], ignore_index=True)

journeysDao.Update(journeys_df, subset=["station_id_from", "station_id_to"])

ValueError: DataFrame must contain columns: {'lon', 'station_id', 'lat'}